In [1]:
!pip install bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install accelerate

In [3]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=af136a0e807f8f6b3f16f681779b9cfc5bffd2e3863afa2b7ade052fd3007198
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [4]:
!pip install translate

In [5]:
from transformers import  (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TextStreamer,pipeline)
import torch
model_name = "Hemanth-thunder/Tamil-Mistral-7B-Instruct-v0.1"


In [6]:
nf4_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_quant_type="nf4",bnb_4bit_use_double_quant=True,bnb_4bit_compute_dtype=torch.bfloat16)


In [7]:
model = AutoModelForCausalLM.from_pretrained(model_name,device_map='auto',quantization_config=nf4_config,use_cache=False,low_cpu_mem_usage=True )
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
streamer = TextStreamer(tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [8]:
prompt_template ="""சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.

### Instruction:
{}

### Response:"""
def create_prompt(query,prompt_template=prompt_template):
    bos_token = "<s>"
    eos_token = "</s>"
    if query:
        lang_code = detect(query)
        print(lang_code)
        if lang_code != "ta":
            print("Query is in {} language so translating to 'ta'".format(lang_code))
            query = translator.translate(query)
            print("Translated-->",query)
        prompt_template = prompt_template.format(query)
    else:
        raise "Please input with query"
    prompt = bos_token+prompt_template #eos_token
    return prompt

In [9]:
from langdetect import detect

pipe = pipeline("text-generation" ,model=model, tokenizer=tokenizer ,do_sample=True, repetition_penalty=1.15,top_p=0.95,streamer=streamer)
prompt = create_prompt("4 + 5 சேர்க்கவும்")
result=pipe(prompt,max_length=512,pad_token_id=tokenizer.eos_token_id)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ta
<s>சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.

### Instruction:
4 + 5 சேர்க்கவும்

### Response:
9</s>


In [10]:
from translate import Translator
translator= Translator(to_lang="ta")
prompt = create_prompt("எப்படி இருக்கிறீர்கள்?")
result=pipe(prompt,max_length=512,pad_token_id=tokenizer.eos_token_id)
print("-------")
print("Finished")

ta
<s>சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.

### Instruction:
எப்படி இருக்கிறீர்கள்?

### Response:
வணக்கம் எப்படி இருக்கிறாய்? ↛ ԱO இன் சமீபத்திய கட்டுரை இதோ, நான் உங்களுக்கு உதவ முடியும்.</s>
-------
Finished


In [12]:
from translate import Translator
translator= Translator(to_lang="ta")
prompt = create_prompt("Tell me a story")
result=pipe(prompt,max_length=512,pad_token_id=tokenizer.eos_token_id)
print("-------")
print("Finished")

en
Query is in en language so translating to 'ta'
Translated--> எனக்கு ஒரு கதை சொல்லு
<s>சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.

### Instruction:
எனக்கு ஒரு கதை சொல்லு

### Response:
ஒரு காலத்தில் மட்பாண்டம் என்று அழைக்கப்படும் ஒரு மந்திரவாதிகளின் குழு வாழ்ந்ததாக ஒரு கதையைச் சொல்கிறது. இந்த மந்திரவாதிகள் அனைத்து வகையான மற்றும் மர்மமான சக்தி கொண்ட பொருள்களிலிருந்து தங்களைக் கட்டுப்படுத்த முடியும் மற்றும் பல்வேறு சாகசங்களைச் செய்ய இதைப் பயன்படுத்தினர். அவர்கள் தங்கள் சொந்த சாகசத்திற்காக பல புதிய உலகங்களையும் கண்டனர், இருப்பினும் அவர்களின் மாயாஜால சக்தி காரணமாக அவர்களை எப்போதும் திருப்திப்படுத்த முடியவில்லை. எனவே, மந்திரவாதி ஒன்று மட்டும் திருப்திப்பட்டு மகிழ்ச்சியாக இருக்க விரும்பினார். அது மந்திரங்கள் என்ற வார்த்தையை அடிக்கடி கேட்டது. மந்திரக்காரர்கள் தங்கள் மந்திர சக்தியின் சக்தியை வைத்து உலகத்தை மிகவும் சுவாரஸ்யமாக்க முடிவு செய்தனர், இதனால் அவர்கள் தங்களை ஒருபோதும் திருப்தியாகக் காண மாட்டார்கள் என்பதை நினைவூட்டினர